### 前（反）向传播
在前向传播过程中，每一层神经元都会接收到前一层神经元的输出作为输入，并通过激活函数进行处理，然后将结果传递给下一层神经元。


反向传播是一种在神经网络中计算参数梯度的方法，它是深度学习中的基础算法。该过程从前向传播开始，计算并存储每一层的输出，然后计算损失函数，接着按照相反的顺序计算每一层的梯度，最后使用这些梯度更新网络参数。在上图中的单隐藏层简单网络的参数是 $W^{1}$ 和 $W^{2}$ 。则反向传播的目的是计算梯度 $\frac{\partial{J}}{\partial{W_{1}}}$ 和 $\frac{\partial{J}}{\partial{W_{2}}}$ 。


反向传播算法在计算每一层的梯度时，需要使用到前向传播中存储的激活值和后续层的梯度值。这些中间结果必须在内存中保留，以便在更新参数时使用。中间值的大小与网络层的数量和批量大小大致成正比，因此，使用更大的批量或更深的网络可能会导致内存不足的问题。也因此，训练比预测需要更多的内存。

### 梯度消失和爆炸
从数学视角分析梯度消失和爆炸的原因：考虑⼀个具有 $L$ 层、输入为 $x$ 和输出为 $o$ 的深层⽹络。每⼀层 $l$ 由变换 $f_{l}$ 定义，该变换的参数为权重 $W^{l}$，对应层的输入变量是 $h^{l}$ （令 $h^{0} = x$ ）。则网络可以表示为：

$$
h^{l} = f_{l}(h^{l-1})         ——  o = f_{L} \circ ··· \circ f_{1}(x)
$$

如果所有隐藏变量和输入都是向量，我们可以将输出变量 $o$ 关于任何⼀组参数 $W^{l}$ 的梯度写为下式：

$$
\frac{\partial{o}}{\partial{W^{l}}} = \frac{\partial{o}}{\partial{h^{L}}} \frac{\partial{h^{L}}}{\partial{h^{L-1}}} ··· \frac{\partial{h^{l+1}}}{\partial{h^{l}}} \frac{\partial{h^{l}}}{\partial{W^{l}}}
$$

- 梯度是 $L-l$ 个矩阵 $M(L)...M(L+1)$ 与输出层的初始梯度 $v^{l}$ 的乘积。当将太多的概率乘在一起时，这些问题经常会出现。初始化之后，矩阵 $M^{l}$ 可能具有各种各样的特征值。他们可能很小，也可能很大。他们的乘积也可能非常大，也可能非常小。这是导致梯度爆炸和梯度消失的共同原因。
- ReLU激活函数有助于减轻梯度消失问题，是因为当输入为正时，其导数恒为1。因此，在正输入区间内，梯度不会随着网络深度的增加而衰减。
- 不稳定梯度威胁到我们优化算法的稳定性。要么是梯度爆炸问题：参数更新过大，破坏了模型的稳定收敛；要么是梯度消失问题：参数更新过小，在每次更新时几乎不会移动，导致模型无法学习。

### BN层：Batch Normalization
在模型训练过程中，批量规范化利用小批量的均值和标准差，不断调整神经网络的中间输出，这样可以使得网络在训练过程中更加稳定，加速收敛，同时也可以一定程度上防止过拟合。

批量规范化的基本思想是对每一层的输入进行规范化处理，使其满足均值为0，方差为1的标准正态分布，然后通过学习参数来恢复网络层需要的原始分布。对于来自小批量 $B$ 的输入 $x$ ，批量规范化 $BN(x)$ 可以按照以下表达式进行：

$$
BN(x) = \gamma \odot \frac{x - \widehat{\mu}_{B}}{\widehat{\sigma}_{B}} + \beta
$$

$$
\widehat{\sigma}_{B} = \frac{1}{|B|} \sum_{x \in B}x 
$$

$$
\widehat{\sigma}^{2}_{B} = \frac{1}{|B|} \sum_{x \in B}(x - \widehat{\mu}_{B})^{2} + \epsilon
$$


其中 $\gamma，\beta$ 是训练过程中的缩放和偏移（引入的可学习参数）；注意到加上 $\epsilon$ 的原因是，在实际操作中，如果批量的方差 ( $\widehat{\sigma}^{2}_{B}$ ) 非常小或者为零（比如，当批量中的所有样本都相等时），直接进行除法会导致除以零的错误。加上一个小的 $\epsilon$ 可以保证分母不会为零。

**实践中BN层的位置**：批量规范化层置于全连接层中的仿射变换和激活函数之间。先规范再激活。

$$
h = \phi(BN(Wx + b))
$$

**预测过程中的BN层**：常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。